# <b><center>Feature Engineering</center></b>

## Import libaries

In [1]:
# ! pip install sentence_transformers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline
# Set the option to display all columns
pd.set_option('display.max_columns', None)

## Import Data

In [3]:
df__bk_rat = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', encoding="latin-1")
df_bk_dt = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX_Books.csv', sep=';', encoding="latin-1")
df_usr_dt = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Users.csv', sep=';', encoding="latin-1")
prepo_df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [4]:
prepo_df.head()

## Extract Features 

### Age Group

In [5]:
# Define the number of age bins
num_age_bins = 5  # You can adjust this number based on your preference

# Create age bins
prepo_df["age_group"] = pd.cut(prepo_df["age"], bins=num_age_bins, labels=False)

In [6]:
prepo_df['age_group'].unique()

### Continet

In [7]:
def get_continent(country):
    # Check for NaN values
    if pd.isna(country):
        return "Other"

    # Define a mapping between continents and some keywords that might appear in country names
    continent_mapping = {
        "North America": ["usa", "canada", "mexico"],
        "Europe": ["europe", "uk", "germany", "france", "spain", "italy"],
        "Asia": ["asia", "china", "japan", "india"],
        "South America": ["south america", "brazil", "argentina"],
        "Africa": ["africa", "nigeria", "egypt", "south africa"],
        "Australia": ["australia"],
        "Antarctica": ["antarctica"]
    }

    # Check if any continent keyword is present in the country name
    country = country.lower()
    for continent, keywords in continent_mapping.items():
        for keyword in keywords:
            if keyword in country:
                return continent

    # If no continent is found, return "Other"
    return "Other"

prepo_df["continent"] = prepo_df["country"].apply(get_continent)

In [8]:
prepo_df.head()

### Books categories 

In [9]:
prepo_df['Category'].nunique()

we need to minimize it

In [10]:
prepo_df.groupby(['Category'])['Category'].count().sort_values()

let's clean commas and puncuations 

In [11]:
prepo_df['Category'] = prepo_df['Category'].str.strip(r'[]\'.,\"')

In [12]:
prepo_df.groupby(['Category'])['Category'].count()

we lowered the number from 6,448 to 6,172. next, lets continue to minimize. how many categories are with more than 1,000 books?

In [13]:
len(prepo_df.groupby(['Category'])['Category'].count()[prepo_df.groupby(['Category'])['Category'].count()>1000])

unite similar categories

In [14]:
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:11]!='Young Adult' else 'Young Adult')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:11]!='Young adult' else 'Young Adult')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:3]!='Zoo' else 'Zoo')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:7]!='Cookery' else 'Cookery')
#create a a higher hierarchy for category
prepo_df['big_Category'] = prepo_df['Category'].copy()
prepo_df['big_Category'] = prepo_df['big_Category'].apply(lambda txt: 'literary' if 'literary' in txt else txt)
prepo_df['big_Category'] = prepo_df['big_Category'].apply(lambda txt: 'fiction' if 'fiction' in txt else txt)

In [15]:
prepo_df['Category'] = prepo_df['Category'].str.lower()

In [16]:
prepo_df.groupby(['Category'])['Category'].count()[prepo_df.groupby(['Category'])['Category'].count()>1000].sort_values(ascending=False)#[df.groupby(['Category'])['Category'].count()>500]

In [17]:
prepo_df.groupby(['Category'])['Category'].count()

#### BERT to caculate categories similarity

In [18]:
# Create a SentenceTransformer model (BERT-based)
model = SentenceTransformer("bert-base-nli-mean-tokens")

In [ ]:
# Encode category names into embeddings
category_embeddings = model.encode(prepo_df["Category"], show_progress_bar=True)

In [ ]:
# Use Agglomerative Clustering to group similar categories
clustering_model = AgglomerativeClustering(n_clusters=10)  # You can adjust the number of clusters
clusters = clustering_model.fit_predict(category_embeddings)

# Assign cluster labels to the DataFrame
df_filtered["grouped_category"] = clusters

### Save to new csv 

In [ ]:
#Save the modified DataFrame back to a CSV file if needed
data.to_csv("processed_data_upd.csv", index=False)